In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/body-signal-of-smoking/smoking.csv')
df.info()

Data'dan gördüğümüz kadarıyla smoking columnunu 1 ve 0 lardan oluşuyor. Yine de testimizi yapalım.
.nuniqiue fonksiyonu ile sadece 2 farklı değeri olduğunu görüyoruz

In [ ]:
df.smoking.nunique()

Bu datamızdaki amacımız sağlık değerlerine sigara kullanımının etkisi var mı ? Yok mu ?
Bu sebeple datamız içindeki yalnızca kullanacağımız futureları alıyoruz.

In [ ]:
dfs = df.iloc[:,10:27]
dfs.head()

Ve bu datamız içinde de yine kullanmayacağımız bölümler var. Drop ile kullanmadığımız futureları atabilirdim
fakat sıralı bi şekilde durdukları için .iloc daha kolay geldi ve kalan 3 future da drop yoluyla çıkardım.

In [ ]:
dfs = dfs.drop(['oral','tartar','Urine protein'],axis=1)
dfs.head()

Şimdi predict edeceğimiz 'smoking' columnunu y değişkenine. Predict etmek için kullanacağımız veriyi ise predict
edeceğimiz future olmadan x_data değişkenine atıyoruz.

In [ ]:
y = dfs.smoking.values
x_data = dfs.drop(['smoking'],axis=1)

print(y.shape,x_data.shape)

Ayırdığımız futurelar içinde çok yüksek ve düşük değerler birlikte bulunuyor. Bunu engellemek için normalize etmemiz
gerekiyor. Yani kısaca futurlarımızı 0 ile 1 arasında değerlere eşliyoruz.

In [ ]:
x = (x_data - np.min(x_data)) / (np.max(x_data) - np.min(x_data)).values

Datamızın bir bölümünü modelimizi train etmek için bir bölümünü ise test için kullanacağız. Sklearn kütüphanesi ile
bu ayrımı yüzde 35'e 65 olarak ayarladım. Bu modelimizin hata payı en düşük olan değerdi deneyerek seçildi.(test_size)
Random state'i ise bu seçilen kısmın random değil her çalıştırıldığında aynı kısım olmasıni istetiğimizden 42 verdik


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.35,random_state=42)

x_train = x_train.T
x_test = x_test.T
y_train = y_train.T
y_test = y_test.T

print("x_train: ",x_train.shape)
print("x_test: ",x_test.shape)
print("y_train: ",y_train.shape)
print("y_test: ",y_test.shape)

Lojistik regresyon modeli formül;
Futurelarımızı deneyerek belirlediğimiz bir değer(weight) ile çarpıyoruz ve ardından bias ile topluyoruz
Elde ettiğimiz sonucu z değişkenine atıyoruz. z değişkenini sigmoid fonksiyona sokuyoruz ve sonucumuz
bize tahmin etmek istediğimiz binary değeri veriyor. (0.8)>0.5 so it is 1

z = (w.t)x + b

Weightimizi ve biasımızı belirliyoruz

In [ ]:
def initialize_weights_and_bias(dimension):
    w = np.full((dimension,1),0.01)
    b = 0.0
    return w,b

z değişkenimizi sigmoid fonksiyona sokuyoruz

In [ ]:
def sigmoid(z):
    y_head = 1/(1 + np.exp(-z))
    return y_head

In [ ]:
def forward_backward_propagation(w,b,x_train,y_train):
    z = np.dot(w.T,x_train) + b
    y_head = sigmoid(z)
    loss = -y_train* np.log(y_head)-(1-y_train)*np.log(1-y_head)
    cost = (np.sum(loss))/x_train.shape[1]
    
    derivative_weight = (np.dot(x_train,((y_head-y_train).T)))/x_train.shape[1] # x_train.shape[1]  is for scaling
    derivative_bias = np.sum(y_head-y_train)/x_train.shape[1]                 # x_train.shape[1]  is for scaling
    gradients = {"derivative_weight": derivative_weight, "derivative_bias": derivative_bias}
    
    return cost,gradients

In [ ]:
def update(w, b, x_train, y_train, learning_rate,number_of_iterarion):
    cost_list = []
    cost_list2 = []
    index = []
    
    # updating(learning) parameters is number_of_iterarion times
    for i in range(number_of_iterarion):
        # make forward and backward propagation and find cost and gradients
        cost,gradients = forward_backward_propagation(w,b,x_train,y_train)
        cost_list.append(cost)
        # lets update
        w = w - learning_rate * gradients["derivative_weight"]
        b = b - learning_rate * gradients["derivative_bias"]
        if i % 100 == 0:
            cost_list2.append(cost)
            index.append(i)
            print ("Cost after iteration %i: %f" %(i, cost))
            
    # we update(learn) parameters weights and bias
    parameters = {"weight": w,"bias": b}
    plt.plot(index,cost_list2)
    plt.xticks(index,rotation='vertical')
    plt.xlabel("Number of Iterarion")
    plt.ylabel("Cost")
    plt.show()
    return parameters, gradients, cost_list

In [ ]:
def predict(w,b,x_test):
    # x_test is a input for forward propagation
    z = sigmoid(np.dot(w.T,x_test)+b)
    Y_prediction = np.zeros((1,x_test.shape[1]))
    # if z is bigger than 0.5, our prediction is sign one (y_head=1),
    # if z is smaller than 0.5, our prediction is sign zero (y_head=0),
    for i in range(z.shape[1]):
        if z[0,i]<= 0.5:
            Y_prediction[0,i] = 0
        else:
            Y_prediction[0,i] = 1

    return Y_prediction

In [ ]:
def logistic_regression(x_train, y_train, x_test, y_test, learning_rate ,  num_iterations):
    # initialize
    dimension =  x_train.shape[0]  # that is 30
    w,b = initialize_weights_and_bias(dimension)
    # do not change learning rate
    parameters, gradients, cost_list = update(w, b, x_train, y_train, learning_rate,num_iterations)
    
    y_prediction_test = predict(parameters["weight"],parameters["bias"],x_test)

    # Print test Errors
    print("test accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_test - y_test)) * 100))
    
logistic_regression(x_train, y_train, x_test, y_test,learning_rate = 5, num_iterations = 850)  

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train.T,y_train.T)
print("test accuracy {}".format(lr.score(x_test.T,y_test.T)))